# Simple GRU network with pretrained vectors for initialization

In [1]:
import sys, os, re, csv, codecs, gc, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

from Tokenizer import Tokenizer
from ZeroMaskedLayer import ZeroMaskedLayer
from AttentionLayer import AttentionLayer

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'{utility_path}crawl-300d-2M.vec'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [4]:
def unicodeToAscii(series):
    return series.apply(lambda s: unicodedata.normalize('NFKC', str(s)))


def multiple_replace(text, adict):
    rx = re.compile('|'.join(map(re.escape, adict)))

    def one_xlat(match):
        return adict[match.group(0)]

    return rx.sub(one_xlat, text)

STOP_WORDS = set(stopwords.words( 'english' ))
# Lowercase, trim, and remove non-letter characters
def normalizeString(series):
    series = unicodeToAscii(series)
    series = series.str.lower()
    series = series.str.replace(r"(\n){1,}", "")
    series = series.str.replace(r"\'", "")
    series = series.str.replace(r"\-", "")
    series = series.str.replace(r"[^0-9a-zA-Z.,!?]+", " ")

    return series


In [5]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [6]:
for df in train, test:
    df["comment_text"] = normalizeString(df["comment_text"])

In [7]:
train.comment_text.sample(1).values[0]

'i will test alternate summaries for greater clarity.'

In [8]:
MAX_FEATURES = 200000
MAX_LEN = 200

tok = Tokenizer(max_features=MAX_FEATURES, max_len=MAX_LEN, tokenizer=wordpunct_tokenize)
X_train = tok.fit_transform(train["comment_text"].astype(str))
X_test = tok.transform(test["comment_text"].astype(str))

print(X_train.shape, X_test.shape)

(159571, 200) (153164, 200)


In [9]:
del train, test

In [10]:
EMBED_SIZE = 300
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

def initialize_embeddings(filename, tokenizer):
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(filename))

    word_index = tokenizer.vocab_idx
    nb_words = min(MAX_FEATURES, len(word_index))
    embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
    for word, i in word_index.items():
        if i >= MAX_FEATURES: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [11]:
embedding_matrix = initialize_embeddings(EMBEDDING_FILE, tok)
print(embedding_matrix.shape)
print(np.mean(embedding_matrix), np.std(embedding_matrix))

(200000, 300)
0.002498661320857451 0.21826940261470665


In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
class GRUClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, gru_dim=150, dense_dim=256, batch_size=128, epochs=2, bidirectional=False, 
                 pool_type='all', initial_weights=None, optimizer='adam' ,verbose=1, out_dim=6, callbacks=None,
                spatial_drop=0.0, dropout=0.0, mask_zero=True, 
                gru_kernel_regularization = 0.0001,
                gru_recurrent_regularization = 0.0001,
                gru_bias_regularization = 0.0001,
                embeddings_regularization = 0.0,
                ):
        
        self.gru_dim = gru_dim
        self.dense_dim = dense_dim
        self.batch_size = batch_size
        self.epochs= epochs
        self.bidirectional = bidirectional
        self.pool_type = pool_type
        self.initial_weights = initial_weights
        self.verbose = verbose
        self.callbacks = callbacks
        self.optimizer = optimizer
        self.out_dim = out_dim
        self.spatial_drop = spatial_drop
        self.dropout = dropout
        self.mask_zero = mask_zero
        self.gru_kernel_regularization = gru_kernel_regularization
        self.gru_recurrent_regularization = gru_recurrent_regularization
        self.gru_bias_regularization = gru_bias_regularization
        self.embeddings_regularization = embeddings_regularization
        
    def _build_model(self):
        inp = Input(shape=(MAX_LEN,))
        emb = Embedding(MAX_FEATURES, 
                        EMBED_SIZE,
                        weights=[self.initial_weights],
                        mask_zero=self.mask_zero,
                        #embeddings_regularizer=regularizers.l2(self.embeddings_regularization),
                        trainable=True)(inp)

        if self.mask_zero:
            emb = ZeroMaskedLayer()(emb)
            
        emb = SpatialDropout1D(self.spatial_drop)(emb)
        if self.bidirectional:
            enc = Bidirectional(CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True, stateful=True,
                                         ))(emb)
            x = enc[0]
            state = enc[1]
        else:
            x, state = CuDNNGRU(int(self.gru_dim), return_sequences=True, return_state=True,
                            kernel_regularizer=regularizers.l2(self.gru_kernel_regularization),
                            recurrent_regularizer=regularizers.l2(self.gru_recurrent_regularization),
                            bias_regularizer=regularizers.l2(self.gru_bias_regularization)
                               )(emb)
            #x = SpatialDropout1D(0.5)(x)
        
        if self.pool_type == 'avg':
            x = GlobalAveragePooling1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'max':
            x = GlobalMaxPool1D()(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'attn':
            x = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x, state])
            
        elif self.pool_type == 'all':
            x1 = GlobalAveragePooling1D()(x)
            x2 = GlobalMaxPool1D()(x)
            x3 = AttentionLayer(MAX_LEN)(x)
            x = concatenate([x2, x3, state])
    
        x = Dropout(self.dropout)(x)
        x = Dense(self.dense_dim)(x)
        x = PReLU()(x)
        
        #x = Dense(self.dense_dim)(x)
        #x = PReLU()(x)

        out = Dense(self.out_dim, activation="sigmoid")(x)
        if self.optimizer == 'adam':
            opt = Adam(lr=0.001, decay=0.0, clipnorm=1.0)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(clipnorm=1.0)
        model = Model(inputs=inp, outputs=out)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return model
    
    def fit(self, X, y):
        self.model = self._build_model()
        
        if self.callbacks:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       callbacks=self.callbacks,
                       shuffle=True)
        else:
            self.model.fit(X, y, batch_size=self.batch_size, epochs=self.epochs,
                       verbose=self.verbose,
                       shuffle=True)
        return self
    
    def predict(self, X, y=None):
        if self.model:
            y_hat = self.model.predict(X, batch_size=1024)
        else:
            raise ValueError("Model not fit yet")
        return y_hat

In [13]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.001
    if epoch == 1:
        return 0.0008
    if epoch == 2:
        return 0.001
    if epoch == 3:
        return 0.00001


def shuffle_crossvalidator(model, cvlist, X, y, lr_decay):
    y_trues = []
    y_preds = []
    scores = []
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        print("ROC AUC for this fold is ", score)
        y_trues.append(y_val)
        y_preds.append(y_pred)
        K.clear_session()
        gc.collect()
        #break
    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    score = roc_auc_score(y_trues, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, scores

def outoffold_crossvalidator(model_params, cvlist, X, y, lr_decay):
    y_preds = np.zeros(y.shape)
    LRDecay = LearningRateScheduler(lr_decay)

    for tr_index, val_index in cvlist2:
        X_tr, y_tr = X[tr_index, :], y[tr_index, :]
        X_val, y_val = X[val_index, :], y[val_index, :]
        RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
        
        model.set_params(**{'callbacks':[RocAuc, LRDecay]})
        model.fit(X_tr, y_tr)

        y_pred = model.predict(X_val)
        print("ROC AUC for this fold is ", roc_auc_score(y_val, y_pred))
        y_preds[val_idx] = y_pred
        K.clear_session()
        break
    score = roc_auc_score(y, y_preds)
    print("Overall score on 10 fold CV is {}".format(score))
    
    return y_preds, y_trues, score


In [14]:
#model = GRUClassifier(gru_dim=300, dense_dim=600, initial_weights=embedding_matrix, bidirectional=False,
#                    batch_size=100, epochs=2, optimizer='adam', pool_type='all')

#y_preds, y_trues, _ = shuffle_crossvalidator(model, cvlist2, X_train, y)

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


def uniform_int(name, lower, upper):
    # `quniform` returns:
    # round(uniform(low, high) / q) * q
    return hp.quniform(name, lower, upper, q=1)

def loguniform_int(name, lower, upper):
    # Do not forget to make a logarithm for the
    # lower and upper bounds.
    return hp.qloguniform(name, np.log(lower), np.log(upper), q=1)

parameter_space = {
    'gru_dim': uniform_int('gru_dim', 50, 600),
    'dense_dim': uniform_int('dense_dim', 100, 1000),
    'lr1': hp.uniform('lr1', 0.0001, 0.005),
    'lr2': hp.uniform('lr2', 0.0001, 0.005),
    'spatial_drop': hp.uniform('spatial_drop', 0, 0.5),
    'dropout': hp.uniform('dropout', 0, 0.5),
    'batch_size': loguniform_int('batch_size', 16, 512),
    'mask_zero': hp.choice('mask_zero', [True, False]),
    'optimizer': hp.choice('optimizer', ['adam', 'rmsprop']),
    'pool_type': hp.choice('pool_type', ['avg', 'max', 'attn', 'all']),
    'bidirectional': hp.choice('bidirectional', [True, False]),
    'gru_kernel_reg': hp.loguniform('gru_kernel_reg', np.log(1e-10), np.log(1e-4)),
    'gru_recc_reg': hp.loguniform('gru_recc_reg', np.log(1e-10), np.log(1e-4)),
    'gru_bias_reg': hp.loguniform('gru_bias_reg', np.log(1e-10), np.log(1e-4)),
    #'embeddings_reg': hp.loguniform('embeddings_reg', 1e-8, 1e-4)
}


def objective(parameter_space):
    
    def lr_decay(epoch):
        if epoch == 0:
            return parameter_space['lr1']
        if epoch == 1:
            return parameter_space['lr2']
    
    model = GRUClassifier(initial_weights=embedding_matrix, bidirectional=False,
                          gru_dim = int(parameter_space['gru_dim']),
                          dense_dim = int(parameter_space['dense_dim']),
                          mask_zero = parameter_space['mask_zero'],
                          pool_type = parameter_space['pool_type'],
                          batch_size= int(parameter_space['batch_size']), 
                          epochs=2, 
                          optimizer=parameter_space['optimizer'],
                          dropout=parameter_space['dropout'],
                          spatial_drop=parameter_space['spatial_drop'],
                          gru_kernel_regularization = parameter_space["gru_kernel_reg"],
                          gru_recurrent_regularization = parameter_space["gru_recc_reg"],
                          gru_bias_regularization = parameter_space["gru_bias_reg"],
                          #embeddings_regularization = parameter_space["embeddings_reg"],
                          )

    y_preds, y_trues, scores = shuffle_crossvalidator(model, cvlist1, X_train, y, lr_decay)    
    score = roc_auc_score(y_trues, y_preds)
    print("Score for parameters {} is {}".format(parameter_space, score))
    #return score
    return {
        'loss': -1* score,
        'status': STATUS_OK,
        'other_Stuff': {'scores': scores, 'variance': np.std(scores)},
        }

trials = Trials()

best = fmin(objective,
    space=parameter_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
           )

print(best)

Epoch 1/2
 18336/151592 [==>...........................] - ETA: 3:51 - loss: 0.0935 - acc: 0.9727

In [ ]:
import gc 
gc.collect()
K.clear_session()